# README
Requires instalation of the workspace package structuredstreamforspark_2.11-2.4.2-1.2.2.jar
[Download Link](https://aad.cosmos08.osdinfra.net/cosmos/cosmos.clientTools/public/Spark/Preview/) (Login with Microsoft account)

This package has not been tested with .NET Spark

In [8]:
from datetime import datetime, timedelta

In [9]:
spark.conf.set("spark.storage.synapse.linkedServiceName", "skypedata_adhoc_c11")
spark.conf.set("fs.azure.sas.token.provider.type", "com.microsoft.azure.synapse.tokenlibrary.LinkedServiceBasedTokenProvider")

In [10]:
# Get the list of files matching the stream set pattern.
# Expected Runtime to search, for existing files, load into dataframe, and display:
# 3 days: 1-4 minutes
# 30 days: (canceled after > 1 hour and no apparent job progression), second try failed after an hour with the error "Column 'cc_participant_1_acs_resource_id' not found in column groups"
reportDate = datetime.utcnow()
# Failed to load 90 days of data with error:
# Job aborted due to stage failure: assertion failed: Column 'MeetingJoinTime' not found in column groups. 
# Cause of failure is likely due to schema drift over time. 
lookback = 7 * 24
searchRange = [reportDate - timedelta(hours=i) for i in range(lookback)]

rootPath = "adl://skypedata-adhoc-c11.azuredatalakestore.net/local/pstnhub_tcdr"
pattern = "{year}/{month}/{day}/tcdr_calls_{year}_{month}_{day}_{hour}.ss"

filepaths = []
for date in searchRange:
    year, month, day, hour = date.strftime("%Y-%m-%d-%H").split('-')
    filename = pattern.format(**locals())
    filepath = f"{rootPath}/{filename}"
    try:
        mssparkutils.fs.ls(filepath)        
        filepaths.append(filepath)
    except:
        print(f"Error: Missing Stream {filepath}")

In [11]:
# The option "filterpushdown" allows for more efficent spark operations.
df = spark.read.format("sstream").option("filterpushdown", True).load(";".join(filepaths))

In [12]:
display(df.sample(0.01))